In [1]:
import dill
import pandas as pd
from sklearn.compose import ColumnTransformer, make_column_selector
import time
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import json
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


In [ ]:
#from sklearn.svm import SVC
from sklearn.tree import 
from sklearn.model_selection import GridSearchCV



df4 = pd.read_csv('data/df3_10k_50n_50p.csv')


df4 = ad_campaign(df4)
df4 = day_of_week(df4)
df4 = empties(df4)
df4 = resolution_func(df4)
df4 = country_v_2(df4)
df4 = city_v_2(df4)
df4 = device_brand(df4)
df4 = encode_stuff(df4)
df4 = scale_stuff(df4)
df4 = filter_stuff(df4)

y = df4['event_action']
x = df4.drop('event_action', axis=1)

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3)




# Define parameters and their possible values in a dictionary
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10]
}

# Create Logistic Regression classifier object
classifier = DecisionTreeClassifier()

# Perform Grid Search with cross-validation (e.g., using k-fold CV)
grid_search = GridSearchCV(classifier,
                           param_grid=param_grid,
                           scoring='roc_auc',
                           cv=5)

# Fit the model on training data and find optimal parameters based on performance metric (default is accuracy)
grid_search.fit(x_train, y_train)

# Get best parameters found during grid search
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Use the best estimator/model for predictions on test data 
y_pred = grid_search.predict(x_test)
print("Best Parameters: ", grid_search.best_params_)
print("Best Score (ROC AUC): ", grid_search.best_score_)

In [2]:
df1 = None

In [3]:
def event_action(df3):
    target_action = ['sub_car_claim_click', 
                 'sub_car_claim_submit_click',
                 'sub_open_dialog_click', 
                 'sub_custom_question_submit_click', 
                 'sub_call_number_click', 
                 'sub_callback_submit_click', 
                 'sub_submit_success', 
                 'sub_car_request_submit_click'
                ]

    df3['event_action'] = df3['event_action'].apply(lambda x: 1 if x in target_action else 0)
    return df3

In [4]:
def sample_df3(df3, total_rows =  200000, neg_percent = 50, pos_percent = 50):
    df3_pos = df3[df3['event_action'] == 1].sample(int(total_rows / 100 * pos_percent))
    df3_neg = df3[df3['event_action'] == 0].sample(int(total_rows / 100 * neg_percent))
    df3_pos = df3_pos.reset_index()
    df3_neg = df3_neg.reset_index()
    df3_pos = df3_pos.drop('index', axis=1)
    df3_neg = df3_neg.drop('index', axis=1)
    df3 = pd.concat([df3_pos, df3_neg])
    return df3

In [5]:
def ad_campaign(df3):
    try:
        with open('data/utm_c_frec_dict1.json', 'r') as f:
            utm_c_frec_dict = json.load(f)
    except FileNotFoundError:
        print("oh.., looks like its the first time you run it - lil' bit longer then, m8. pls hold:)")

        utm_c_frec_dict = {}
        counter = 1
        for pos in df3.utm_campaign.unique():
            if len(df3[(df3.utm_campaign == pos) & (df3.event_action == 1)]) == 0:
                utm_c_frec_dict[str(pos)] = 0
            else:
                utm_c_frec_dict[str(pos)] = round(len(df3[(df3.utm_campaign == pos) & (df3.event_action == 1)]) / len(df3[df3.utm_campaign == pos]), 5)

            #print(counter)
            counter = counter  + 1
        with open('data/utm_c_frec_dict1.json', 'w') as f: 
            json.dump(utm_c_frec_dict, f)

    finally:
        df3['camp_succ_rate'] = df3.utm_campaign.apply(lambda x: utm_c_frec_dict[str(x)])
    return df3

In [6]:
def day_of_week(df3):
    df3['new_date'] = pd.to_datetime(df3['visit_date'])
    df3['day_of_week'] = df3.new_date.dt.dayofweek
    return df3

In [7]:
def empties(df3):
    df3.loc[df3.utm_source.isna() == True, 'utm_source'] = 'other'
    df3.loc[df3.utm_adcontent.isna() == True, 'utm_adcontent'] = 'Other'
    df3.loc[df3.device_brand.isna() == True, 'device_brand'] = 'other'
    return df3

In [8]:
def resolution_func(df3):
    #resolution
    bounds = []
    df3['resolution'] = df3.device_screen_resolution.apply(lambda x:eval(x.replace('x','*')))
    for device in df3.device_category.unique():
        q25 = df3[df3.device_category == device].resolution.quantile(0.25)
        q75 = df3[df3.device_category == device].resolution.quantile(0.75)
        iqr = q75 - q25
        bounds.append((device, q25 - 1.5 * iqr, q75 + 1.5 * iqr))




    test_list = list(df3.device_screen_resolution)
    test_list2 = list(df3.device_category)

    for i in range(len(test_list)):
        test_list[i] = eval(test_list[i].replace('x','*'))

    tst_l = list(zip(test_list2, test_list))

    resolution = []

    for i in range(len(tst_l)):
        if tst_l[i][0] == bounds[0][0]:
            resolution.append(bounds[0][0]+'_high' if tst_l[i][1] >= bounds[0][2] * 0.7 else (bounds[0][0]+'_medium' if bounds[0][2] * 0.7 > tst_l[i][1] >= bounds[0][2] * 0.1 else bounds[0][0]+'_low'))
        elif tst_l[i][0] == bounds[1][0]:
            resolution.append(bounds[1][0]+'_high' if tst_l[i][1] >= bounds[1][2] * 0.7 else (bounds[1][0]+'_medium' if bounds[1][2] * 0.7 > tst_l[i][1] >= bounds[1][2] * 0.1 else bounds[1][0]+'_low'))
        elif tst_l[i][0] == bounds[2][0]:
            resolution.append(bounds[2][0]+'_high' if tst_l[i][1] >= bounds[2][2] * 0.7 else (bounds[2][0]+'_medium' if bounds[2][2] * 0.7 > tst_l[i][1] >= bounds[2][2] * 0.1 else bounds[2][0]+'_low'))

    df3['device_screen_resolution'] = resolution
    df3 = df3.drop('resolution', axis=1)
    
    return df3
    

In [9]:
def country(df3, trsh = 0.001):
    #geo_country
    country_list = list(df3.geo_country.unique())
    for i in range(len(country_list)):
        country_list[i] = ( len(df3[df3.geo_country == country_list[i]]), country_list[i])
    country_list = sorted(country_list, reverse=True)

#    trsh = 0.0005
    df3_len = len(df3) 
    for item in country_list:
        if len(df3[df3.geo_country == item[1]]) / df3_len >= trsh:
            continue
        else:
            df3.loc[df3.geo_country == item[1], 'geo_country'] = 'some_unimportant_country'
    return df3

In [10]:
def city(df3, trsh = 0.02):
    #geo_city
    city_list = []
    df3_len = len(df3)
    try:
        with open('data/city_list1.txt', 'r') as f:
            for line in f:
                # remove newline character and parentheses
                line = line.rstrip('\n').replace('(', '').replace(')', '').replace("'", '')
                # split on comma and convert each element to correct type
                tuple_elements = [int(e.strip()) if e.strip().isdigit() else e.strip() for e in line.split(',')]
                # create tuple and add to list
                my_tuple = tuple(tuple_elements)
                city_list.append(my_tuple)


    except FileNotFoundError:
        print("oh.., looks like its the first time you run it - lil' bit longer then, m8. pls hold:)")

        city_list = list(df3.geo_city.unique())
        for item in range(len(city_list)):
            city_list[item] =  (len(df3[df3.geo_city == city_list[item]]), city_list[item])
        city_list = sorted(city_list, reverse=True)

        with open('data/city_list1.txt', 'w') as f:
            for t in city_list:
                f.write(str(t) +'\n')



    finally:
#        trsh = 0.0005
        city_list_valid = []
        for item in city_list:
            #print(item[1], ' - ', round(item[0] / df3_len, 4),'%' )
            if round(len(df3[df3.geo_city == item[1]]) / df3_len, 4) >= trsh:
                city_list_valid.append(item[1])
                #print('trsh == 2000 - ', item[0], item[1], round(item[0] / df3_len, 4) >= trsh, ' - appended')

        df3.loc[(~df3['geo_city'].isin(city_list_valid)), 'geo_city'] = 'some_unimportant_city'
    
    return df3

In [11]:
def encode_stuff(df):
    cols_to_encode = ['utm_source', 
                      'utm_medium', 
                      'utm_adcontent', 
                      'device_brand', 
                      'device_category', 
                      'device_screen_resolution', 
                      'device_browser',
                      'utm_campaign',
                      'geo_country',
                      'geo_city'
                     ]
    #encoding
    encoded_features = pd.DataFrame()

    for col in cols_to_encode:

        pre_encoded_df = df[[col]]
        encoder = OneHotEncoder(categories='auto', handle_unknown='ignore', sparse=False)
        encoded_array = encoder.fit_transform(pre_encoded_df)
        #feature_names = [f'{col}_{name}' for name in encoder.get_feature_names_out()]
        feature_names = encoder.get_feature_names_out()
        encoded_df = pd.DataFrame(encoded_array, columns=feature_names)

        if len(encoded_features) == 0:
            encoded_features = encoded_df.copy()
        else:
            encoded_features[feature_names] = encoded_df.values


    df = df.join(encoded_features)
    return df

In [12]:
def scale_stuff(df3):
    #scaling
    cols_to_scale = ['visit_number']

    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df3.loc[:,cols_to_scale])
    scaled_feature_names = [f'{name}_scaled' for name in scaler.get_feature_names_out()]
    #scaler.get_feature_names_out()

    scaled_df = pd.DataFrame(scaled_features, columns=scaled_feature_names)

    df3 = df3.join(scaled_df)

    df3 = df3.drop('visit_number', axis=1)
    return df3

In [41]:
def filter_stuff(df3):
    #pre-existing list of columns
    df_columns = [
        'session_id',
        'hit_date',
        'hit_time',
        'hit_number',
        'hit_type',
        'hit_referer',
        'hit_page_path',
        'event_category',
        'event_label',
        'event_value'
    ]
    
    cols_to_encode = [
        'utm_source',
        'utm_medium', 
        'utm_adcontent',
        'device_brand', 
        'device_category', 
        'device_screen_resolution',
        'device_browser',
        'utm_campaign',
        'geo_country',
        'geo_city'
    ]
    #dropping
    cols_to_drop = []
    for col in df_columns:
        cols_to_drop.append(str(col))
    cols_to_drop = cols_to_drop + ['client_id','new_date', 'visit_date', 'utm_keyword', 'device_os', 'device_model', 'visit_time']    
    df3 = df3.drop(cols_to_drop, axis=1)
    df3 = df3.drop(cols_to_encode, axis=1)
    return df3

In [14]:
def check_stuff(df3):
    #checking

    counter = 0
    for feature in df3.columns:
        if df3[feature].dtype != 'O':
            #print(feature, ' - ', df3[feature].dtype)
            counter += 1
        else:
            print(feature)
    print(counter == len(df3.columns))


    #checking 2
    counter = 0
    for feature in df3.columns:
        if len(df3[df3[str(feature)].isna() == True]) != 0:
            print(feature, ' - ', len(df3[df3[str(feature)].isna() == True]))
            counter += 1




    if counter == 0:
        print('vse zaebis", pustukh fi4ei net')    
    print(df3.shape)
    pass

In [15]:
with open('data/best_params_rf_10k_fit.json', 'r') as f:
    best_params = json.load(f)
best_params

{'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 400}

In [16]:
def predict_stuff(df3):
    y = df3['event_action']
    
    df3 = df3.drop('event_action', axis=1)
    
    x_train, x_test, y_train, y_test = train_test_split(df3,y, test_size=0.3)
    
    rf = RandomForestClassifier(n_estimators=400, min_samples_leaf=2, max_features='sqrt')
    rf.fit(x_train, y_train)
    
    predicted_train = rf.predict(x_train)
    predicted_test = rf.predict(x_test)
    
    print('train acc score - ',accuracy_score(y_train, predicted_train))
    print('test acc score - ', accuracy_score(y_test, predicted_test))

    print('train roc score - ',roc_auc_score(y_train, rf.predict_proba(x_train)[:,1]))
    print('test roc score - ',roc_auc_score(y_test, rf.predict_proba(x_test)[:,1]))
    pass


with open('models/rf_model.pkl', 'wb') as file:
    dill.dump(rf, file)

In [ ]:
from './'

In [118]:
df4 = pd.read_csv('data/df3_200k_50n_50p.csv')
df4 = df4.drop('Unnamed: 0', axis=1)
#df4 = event_action(df4)
#df4 = sample_df3(df4)#, 100000, 50, 50)
#print(df4.columns)

#y = df4['event_action']
#df4 = df4.drop('event_action', axis=1)

#df4 = ad_campaign(df4)
df4 = day_of_week(df4)
df4 = empties(df4)
df4 = resolution_func(df4)
df4 = country(df4, 0.001)
df4 = city(df4, 0.02)
df4 = encode_stuff(df4)
df4 = scale_stuff(df4)
df4 = filter_stuff(df4)
check_stuff(df4)
#for i in df4.columns:
 #   print(i)
predict_stuff(df4)

#rf = RandomForestClassifier(n_estimators=400, min_samples_leaf=2, max_features='sqrt')
#score = cross_val_score(rf, X, y, cv=5, scoring='accuracy')#, error_score='raise')
#print(score)

Index(['session_id', 'hit_date', 'hit_time', 'hit_number', 'hit_type',
       'hit_referer', 'hit_page_path', 'event_category', 'event_action',
       'event_label', 'event_value', 'client_id', 'visit_date', 'visit_time',
       'visit_number', 'utm_source', 'utm_medium', 'utm_campaign',
       'utm_adcontent', 'utm_keyword', 'device_category', 'device_os',
       'device_brand', 'device_model', 'device_screen_resolution',
       'device_browser', 'geo_country', 'geo_city', 'target_action'],
      dtype='object')


C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times,

C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

...

C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_11360\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

True
vse zaebis", pustukh fi4ei net
(200000, 777)
train acc score -  0.6648714285714286
test acc score -  0.5433833333333333
train roc score -  0.730867954195557
test roc score -  0.564995833865885


with open('models/rf_model.pkl', 'rb') as f:
    rf = dill.load(f)

In [19]:
df4 = pd.read_csv('data/df3_200k_50n_50p.csv')
df4 = df4.drop('Unnamed: 0', axis=1)
df4 = event_action(df4)
#df4 = sample_df3(df4)#, 100000, 50, 50)
#print(df4.columns)
#y = df4['event_action']
#df4 = df4.drop('event_action', axis=1)
#df4 = ad_campaign(df4)
df4 = day_of_week(df4)
df4 = empties(df4)
df4 = resolution_func(df4)
df4 = country(df4, 0.001)
df4 = city(df4, 0.02)
df4 = encode_stuff(df4)
df4 = scale_stuff(df4)
df4 = filter_stuff(df4)

y = df4['event_action']

df3 = df4.drop('event_action', axis=1)

x_train, x_test, y_train, y_test = train_test_split(df3,y, test_size=0.3)

#rf = RandomForestClassifier(n_estimators=400, min_samples_leaf=2, max_features='sqrt')
#rf.fit(x_train, y_train)

predicted_train = rf.predict(x_train)
predicted_test = rf.predict(x_test)

print('train acc score - ',accuracy_score(y_train, predicted_train))
print('test acc score - ', accuracy_score(y_test, predicted_test))

print('train roc score - ',roc_auc_score(y_train, rf.predict_proba(x_train)[:,1]))
print('test roc score - ',roc_auc_score(y_test, rf.predict_proba(x_test)[:,1]))


C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, 

C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

...

C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_2924\1146768601.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- device_brand_China Phone
- device_brand_Elephone
- device_brand_Essential
- device_brand_Kingplay
- device_brand_Kyocera
- ...
Feature names seen at fit time, yet now missing:
- camp_succ_rate
- device_brand_Fairphone
- device_brand_Gionee
- device_brand_Guophone
- device_brand_Kata
- ...


In [83]:

#f_transformer = ColumnTransformer(transformers=[
 #   ('numerical', numerical_transformer, make_column_selector(dtype_include=['int64', 'float64'])),
  #  ('categorical', categorical_transformer, make_column_selector(dtype_include='object'))
#])
df4 = pd.read_csv('data/df3_200k_50n_50p.csv')

df4 = event_action(df4)

df4 = sample_df3(df4)


y = df4['event_action']
X = df4.drop('event_action', axis=1)

preprocessor = Pipeline(steps=[
    ('target', FunctionTransformer(event_action)),
    ('sampling', FunctionTransformer(sample_df3)),
    ('ad_campaign_feature_creating', FunctionTransformer(ad_campaign)),
    ('day_of_week', FunctionTransformer(day_of_week)),
    ('empties', FunctionTransformer(empties)),
    ('resolution_func', FunctionTransformer(resolution_func)),
    ('country', FunctionTransformer(country)),
    ('city', FunctionTransformer(city)),
    ('encode_stuff', FunctionTransformer(encode_stuff)),
    ('scale_stuff', FunctionTransformer(scale_stuff)),
    ('filter_stuff', FunctionTransformer(filter_stuff)),
    ('check_stuff', FunctionTransformer(check_stuff))
    ])

rf = RandomForestClassifier(n_estimators=400, min_samples_leaf=2, max_features='sqrt')


high_score = 0


pipe = Pipeline(steps=[
('preprocessor', preprocessor),
('classifier', rf)
])
score = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')#, error_score='raise')
print(f'model: {type(model).__name__}, acc_mean: {score.mean():.4f}, acc_std: {score.std():.4f}')
if high_score < score.mean():
    high_score = score.mean()
    cars_pipe = pipe

else:
    continue


#cars_pipe.fit(X, y)


#with open('data/cars_pipe.pkl', 'wb') as file:
#    dill.dump({
#        'model': cars_pipe,
#        'metadata': {
#            'name': 'car price prediction',
#           'author': 'collaborative ffs by this point can i really write myself here yet?)',
#           'version': 0.00000000000000000001,
#            'date': datetime.now(),
#            'type': type(cars_pipe.named_steps["classifier"]).__name__,
#           'accuracy': high_score
#       }
#   }, file)

(200000, 30)
(200000, 30)
(200000, 30)
session_id
hit_date
hit_type
hit_referer
hit_page_path
event_category
event_action
event_label
visit_date
visit_time
utm_source
utm_medium
utm_campaign
utm_adcontent
utm_keyword
device_category
device_os
device_brand
device_model
device_screen_resolution
device_browser
geo_country
geo_city
False
hit_time  -  99169
hit_referer  -  57196
event_label  -  25835
event_value  -  160000
utm_source  -  11
utm_campaign  -  24059
utm_adcontent  -  29663
utm_keyword  -  99650
device_os  -  99104
device_brand  -  39751
device_model  -  158809
(160000, 29)
session_id
hit_date
hit_type
hit_referer
hit_page_path
event_category
event_action
event_label
visit_date
visit_time
utm_source
utm_medium
utm_campaign
utm_adcontent
utm_keyword
device_category
device_os
device_brand
device_model
device_screen_resolution
device_browser
geo_country
geo_city
False
hit_time  -  99225
hit_referer  -  57102
event_label  -  25787
event_value  -  160000
utm_source  -  11
utm_campai

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 345, in fit
    X, y = self._validate_data(
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\base.py", line 565, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\user\anaconda3\lib\site-packages\sklearn\util

In [3]:
#opening & target featuring
df = pd.read_csv('data/ga_hits.csv')
df2 = pd.read_csv('data/ga_sessions.csv')
df3 = pd.merge(df, df2, on='session_id')

target_action = ['sub_car_claim_click', 
                 'sub_car_claim_submit_click',
                 'sub_open_dialog_click', 
                 'sub_custom_question_submit_click', 
                 'sub_call_number_click', 
                 'sub_callback_submit_click', 
                 'sub_submit_success', 
                 'sub_car_request_submit_click'
                ]

df3['event_action'] = df3['event_action'].apply(lambda x: 1 if x in target_action else 0)


#sampling
df3_pos = df3[df3['event_action'] == 1].sample(100000)
df3_neg = df3[df3['event_action'] == 0].sample(100000)
df3_pos = df3_pos.reset_index()
df3_neg = df3_neg.reset_index()
df3_pos = df3_pos.drop('index', axis=1)
df3_neg = df3_neg.drop('index', axis=1)
df3 = pd.concat([df3_pos, df3_neg])


C:\Users\user\AppData\Local\Temp\ipykernel_7132\2800840019.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('data/ga_sessions.csv')


In [4]:
#df3.to_csv('data/df3_200k_50n_50p.csv')

In [5]:

##cleaning & NaN-removal & engeneering

#ad campaingn success rate
try:
    with open('data/utm_c_frec_dict1.json', 'r') as f:
        utm_c_frec_dict = json.load(f)
except FileNotFoundError:
    print("oh.., looks like its the first time you run it - lil' bit longer then, m8. pls hold:)")
    
    utm_c_frec_dict = {}
    counter = 1
    for pos in df3.utm_campaign.unique():
        if len(df3[(df3.utm_campaign == pos) & (df3.event_action == 1)]) == 0:
            utm_c_frec_dict[str(pos)] = 0
        else:
            utm_c_frec_dict[str(pos)] = round(len(df3[(df3.utm_campaign == pos) & (df3.event_action == 1)]) / len(df3[df3.utm_campaign == pos]), 5)
            
        #print(counter)
        counter = counter  + 1
    with open('data/utm_c_frec_dict1.json', 'w') as f: 
        json.dump(utm_c_frec_dict, f)
    
finally:
    df3['camp_succ_rate'] = df3.utm_campaign.apply(lambda x: utm_c_frec_dict[str(x)])

#date to day of week    
df3['new_date'] = pd.to_datetime(df3['visit_date'])
df3['day_of_week'] = df3.new_date.dt.dayofweek


df3.loc[df3.utm_source.isna() == True, 'utm_source'] = 'other'
df3.loc[df3.utm_adcontent.isna() == True, 'utm_adcontent'] = 'Other'
df3.loc[df3.device_brand.isna() == True, 'device_brand'] = 'other'





cols_to_encode = ['utm_source', 
                  'utm_medium', 
                  'utm_adcontent', 
                  'device_brand', 
                  'device_category', 
                  'device_screen_resolution', 
                  'device_browser',
                  'utm_campaign',
                  'geo_country',
                  'geo_city'
                 ]

In [6]:
#resolution
bounds = []
df3['resolution'] = df3.device_screen_resolution.apply(lambda x:eval(x.replace('x','*')))
for device in df3.device_category.unique():
    q25 = df3[df3.device_category == device].resolution.quantile(0.25)
    q75 = df3[df3.device_category == device].resolution.quantile(0.75)
    iqr = q75 - q25
    bounds.append((device, q25 - 1.5 * iqr, q75 + 1.5 * iqr))
           



test_list = list(df3.device_screen_resolution)
test_list2 = list(df3.device_category)

for i in range(len(test_list)):
    test_list[i] = eval(test_list[i].replace('x','*'))

tst_l = list(zip(test_list2, test_list))

resolution = []

for i in range(len(tst_l)):
    if tst_l[i][0] == bounds[0][0]:
        resolution.append(bounds[0][0]+'_high' if tst_l[i][1] >= bounds[0][2] * 0.7 else (bounds[0][0]+'_medium' if bounds[0][2] * 0.7 > tst_l[i][1] >= bounds[0][2] * 0.1 else bounds[0][0]+'_low'))
    elif tst_l[i][0] == bounds[1][0]:
        resolution.append(bounds[1][0]+'_high' if tst_l[i][1] >= bounds[1][2] * 0.7 else (bounds[1][0]+'_medium' if bounds[1][2] * 0.7 > tst_l[i][1] >= bounds[1][2] * 0.1 else bounds[1][0]+'_low'))
    elif tst_l[i][0] == bounds[2][0]:
        resolution.append(bounds[2][0]+'_high' if tst_l[i][1] >= bounds[2][2] * 0.7 else (bounds[2][0]+'_medium' if bounds[2][2] * 0.7 > tst_l[i][1] >= bounds[2][2] * 0.1 else bounds[2][0]+'_low'))

df3['device_screen_resolution'] = resolution
df3 = df3.drop('resolution', axis=1)

In [8]:
#geo_country
country_list = list(df3.geo_country.unique())
for i in range(len(country_list)):
    country_list[i] = ( len(df3[df3.geo_country == country_list[i]]), country_list[i])
country_list = sorted(country_list, reverse=True)

trsh = 0.0005
df3_len = len(df3) 
for item in country_list:
    if len(df3[df3.geo_country == item[1]]) / df3_len >= trsh:
        continue
    else:
        df3.loc[df3.geo_country == item[1], 'geo_country'] = 'some_unimportant_country'

In [9]:
#geo_city
city_list = []
try:
    with open('data/city_list1.txt', 'r') as f:
        for line in f:
            # remove newline character and parentheses
            line = line.rstrip('\n').replace('(', '').replace(')', '').replace("'", '')
            # split on comma and convert each element to correct type
            tuple_elements = [int(e.strip()) if e.strip().isdigit() else e.strip() for e in line.split(',')]
            # create tuple and add to list
            my_tuple = tuple(tuple_elements)
            city_list.append(my_tuple)
            
            
except FileNotFoundError:
    print("oh.., looks like its the first time you run it - lil' bit longer then, m8. pls hold:)")
    
    city_list = list(df3.geo_city.unique())
    for item in range(len(city_list)):
        city_list[item] =  (len(df3[df3.geo_city == city_list[item]]), city_list[item])
    city_list = sorted(city_list, reverse=True)
    
    with open('data/city_list1.txt', 'w') as f:
        for t in city_list:
            f.write(str(t) +'\n')

   
    
finally:
    trsh = 0.0005
    city_list_valid = []
    for item in city_list:
        #print(item[1], ' - ', round(item[0] / df3_len, 4),'%' )
        if round(item[0] / df3_len, 4) >= trsh:
            city_list_valid.append(item[1])

    df3.loc[(~df3['geo_city'].isin(city_list_valid)), 'geo_city'] = 'some_unimportant_city'

In [87]:
counter = 0
for i in cols_to_encode:
    counter += len(df4[str(i)].unique())
counter

662

In [11]:
#encoding
encoded_features = pd.DataFrame()

for col in cols_to_encode:

    pre_encoded_df = df3[[col]]
    encoder = OneHotEncoder(categories='auto', handle_unknown='ignore', sparse=False)
    encoded_array = encoder.fit_transform(pre_encoded_df)
    #feature_names = [f'{col}_{name}' for name in encoder.get_feature_names_out()]
    feature_names = encoder.get_feature_names_out()
    encoded_df = pd.DataFrame(encoded_array, columns=feature_names)
    
    if len(encoded_features) == 0:
        encoded_features = encoded_df.copy()
    else:
        encoded_features[feature_names] = encoded_df.values
        
        
df3 = df3.join(encoded_features)

C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, w

C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performanc

C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performanc

C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performanc

C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performanc

...

C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performanc

C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performanc

C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performanc

C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performanc

C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_features[feature_names] = encoded_df.values
C:\Users\user\AppData\Local\Temp\ipykernel_7132\712945716.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performanc

In [12]:
#scaling
cols_to_scale = ['visit_number']

scaler = StandardScaler()
scaled_features = scaler.fit_transform(df3.loc[:,cols_to_scale])
scaled_feature_names = [f'{name}_scaled' for name in scaler.get_feature_names_out()]
#scaler.get_feature_names_out()

scaled_df = pd.DataFrame(scaled_features, columns=scaled_feature_names)

df3 = df3.join(scaled_df)

df3 = df3.drop('visit_number', axis=1)

In [14]:
#dropping
cols_to_drop = []
for col in df.columns:
    cols_to_drop.append(str(col))
cols_to_drop = cols_to_drop + ['client_id','new_date', 'visit_date', 'utm_keyword', 'device_os', 'device_model', 'visit_time']    
df3 = df3.drop(cols_to_drop, axis=1)
df3 = df3.drop(cols_to_encode, axis=1)


In [25]:
#checking

counter = 0
for feature in df3.columns:
    if df3[feature].dtype != 'O':
        #print(feature, ' - ', df3[feature].dtype)
        counter += 1
    else:
        print(feature)
print(counter == len(df3.columns))


#checking 2
counter = 0
for feature in df3.columns:
    if len(df3[df3[str(feature)].isna() == True]) != 0:
        print(feature, ' - ', len(df3[df3[str(feature)].isna() == True]))
        counter += 1
    
        
    
        
if counter == 0:
            print('vse zaebis", pustukh fi4ei net')    

True
vse zaebis", pustukh fi4ei net


df3.to_parquet('data/df_prepped_200k_parq_50_50.parquet', index=False)

In [ ]:
df3 = pd.read_parquet('data/df_prepped_200k_parq_50_50.parquet')

In [26]:
y = df3['event_action']
df3 = df3.drop('event_action', axis=1)

In [27]:
x_train, x_test, y_train, y_test = train_test_split(df3,y, test_size=0.3)

In [28]:
with open('data/best_params_rf_10k_fit.json', 'r') as f:
    best_params = json.load(f)

In [29]:
best_params

{'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 400}

In [30]:
rf = RandomForestClassifier(n_estimators=400, min_samples_leaf=2, max_features='sqrt')

In [31]:
rf.fit(x_train, y_train)

RandomForestClassifier(min_samples_leaf=2, n_estimators=400)

In [32]:
predicted_train = rf.predict(x_train)
predicted_test = rf.predict(x_test)
print('train acc score - ',accuracy_score(y_train, predicted_train))
print('test acc score - ',printuracy_score(y_test, predicted_test))

print('train roc score - ',roc_auc_score(y_train, rf.predict_proba(x_train)[:,1]))
print('test roc score - ',roc_auc_score(y_test, rf.predict_proba(x_test)[:,1]))



train acc score -  0.8982857142857142
test acc score -  0.88615
train roc score -  0.9380408574759702
test roc score -  0.9088685918785773


In [ ]:
with open('models/rf_model.pkl', 'wb') as file:
    dill.dump(rf, file)